In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 24 13:02:32 2021

@author: lenovo
"""
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.io
import math
import matplotlib.gridspec as gridspec
from plotting import newfig
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import layers, activations
from scipy.interpolate import griddata
from eager_lbfgs import lbfgs, Struct
from pyDOE import lhs

from generate_dataset import *
from utilities import *

OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0-15
OMP: Info #216: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #157: KMP_AFFINITY: 16 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #287: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #287: KMP_AFFINITY: topology layer "L3 cache" is equivalent to "socket".
OMP: Info #287: KMP_AFFINITY: topology layer "L2 cache" is equivalent to "core".
OMP: Info #287: KMP_AFFINITY: topology layer "L1 cache" is equivalent to "core".
OMP: Info #192: KMP_AFFINITY: 1 socket x 8 cores/socket x 2 threads/core (8 total cores)
OMP: Info #218: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 8 maps to socket 0 core 0 thread 1 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 1 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 9 maps to socket 0 core 1 thread 1 
OMP: Info

In [2]:


layer_sizes = [3, 20, 20, 20, 20, 20, 20, 20, 3]
sizes_w = []
sizes_b = []
for i, width in enumerate(layer_sizes):
    if i != 1:
        sizes_w.append(int(width * layer_sizes[1]))
        sizes_b.append(int(width if i != 0 else layer_sizes[1]))



# L-BFGS weight getting and setting from https://github.com/pierremtb/PINNs-TF2.0

def set_weights(model, w, sizes_w, sizes_b):  # Reset parameters

    for i, layer in enumerate(model.layers[1:len(sizes_w) + 1]):
        start_weights = sum(sizes_w[:i]) + sum(sizes_b[:i])
        end_weights = sum(sizes_w[:i + 1]) + sum(sizes_b[:i])
        weights = w[start_weights:end_weights]
        w_div = int(sizes_w[i] / sizes_b[i])
        weights = tf.reshape(weights, [w_div, sizes_b[i]])
        biases = w[end_weights:end_weights + sizes_b[i]]
        weights_biases = [weights, biases]
        layer.set_weights(weights_biases)


def get_weights(model):
    w = []
    for layer in model.layers[1:len(sizes_w) + 1]:
        weights_biases = layer.get_weights()
        weights = weights_biases[0].flatten()
        biases = weights_biases[1]
        w.extend(weights)
        w.extend(biases)
    w = tf.convert_to_tensor(w)
    return w

# def xavier_init(layer_sizes):
#     in_dim = layer_sizes[0]
#     out_dim = layer_sizes[1]
#     xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
#     return tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), dtype=tf.float32)

def neural_net(layer_sizes):

    input_tensor = keras.Input(shape=(layer_sizes[0],))

    hide_layer_list = []
    flag = True
    for width in layer_sizes[1:-1]:
        if flag:
            x = layers.Dense(
                width, activation=tf.nn.tanh,
                kernel_initializer="glorot_normal")(input_tensor)
            flag = False
        else:
            x = layers.Dense(
                width, activation=tf.nn.tanh,
                kernel_initializer="glorot_normal")(x)
    output_tensor = layers.Dense(layer_sizes[-1], activation=None,kernel_initializer="glorot_normal")(x)
    print("xxxxxxxxxxxxxx")
    output0 = output_tensor[:, 0:1]
    output1 = output_tensor[:, 1:2]
    output2 = output_tensor[:, 2:3]

    model_output = keras.models.Model(input_tensor, [output0, output1, output2])

    return model_output

In [ ]:

# initialize the NN
u_model = neural_net(layer_sizes)
# view the NN
u_model.summary()


In [ ]:

@tf.function
def f_model(x, y, t):

    mu = 0.00345
    density = 1056
    u, v, p = u_model(tf.concat([x, y, t],1))

    u_t = tf.gradients(u, t)[0]
    u_x = tf.gradients(u, x)[0]
    u_y = tf.gradients(u, y)[0]
    u_xx = tf.gradients(u_x, x)[0]
    u_yy = tf.gradients(u_y, y)[0]

    v_t = tf.gradients(v, t)[0]
    v_x = tf.gradients(v, x)[0]
    v_y = tf.gradients(v, y)[0]
    v_xx = tf.gradients(v_x, x)[0]
    v_yy = tf.gradients(v_y, y)[0]

    p_x = tf.gradients(p, x)[0]
    p_y = tf.gradients(p, y)[0]

    f_u = u_t + (u * u_x + v * u_y ) + 1.0/density * p_x - mu *(u_xx + u_yy )
    f_v = v_t + (u * v_x + v * v_y ) + 1.0/density * p_y - mu *(v_xx + v_yy )
    div = u_x + v_y 
        
    return f_u, f_v, div

@tf.function
def u_x_model(x, y, t):
    u, v, w = u_model(tf.concat([x, y, t], 1))
    return u, v, w

In [3]:



# define the loss
def loss(x_f_batch, y_f_batch, t_f_batch, xb, yb, tb, ub, vb, weight_ub,  weight_fu):

    f_u_pred, f_v_pred, div_pred = f_model(x_f_batch, y_f_batch, t_f_batch)


    u_pred, v_pred, p_pred = u_model(tf.concat([xb, yb, tb], 1))
    mse_b = 100*weight_ub*(tf.reduce_sum(tf.square(u_pred - ub)) + tf.reduce_sum(tf.square(v_pred - vb)))
    mse_f = weight_fu*(tf.reduce_sum(tf.square(f_u_pred)) + tf.reduce_sum(tf.square(f_v_pred)) + tf.reduce_sum(tf.square(div_pred)))

    return mse_b + mse_f, mse_b, mse_f

xxxxxxxxxxxxxx
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 20)           80          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 20)           420         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 20)           420         dense_1[0][0]                    
_______________________________________________________________________________

In [5]:

@tf.function
def grad(u_model, x_f_batch, y_f_batch, t_f_batch, xb_batch, yb_batch, tb_batch, ub_batch, vb_batch, weight_ub,
         weight_fu):
    with tf.GradientTape(persistent=True) as tape:

        loss_value, mse_b, mse_f = loss(x_f_batch, y_f_batch, t_f_batch, xb_batch, yb_batch, tb_batch, ub_batch,
                                        vb_batch, weight_ub, weight_fu)
        grads = tape.gradient(loss_value, u_model.trainable_variables)

        grads_ub = tape.gradient(loss_value, weight_ub)

        grads_fu = tape.gradient(loss_value, weight_fu)

    return loss_value, mse_b, mse_f, grads, grads_ub, grads_fu


In [6]:



def fit( collo, inlet, outlet, wall, initial , weight_ub, weight_fu , tf_iter, tf_iter2, newton_iter1, newton_iter2):

    batch_sz = N_f
    n_batches = N_f // batch_sz

    start_time = time.time()

    tf_optimizer = tf.keras.optimizers.Adam(lr=0.005, beta_1=.99)
    tf_optimizer_weights = tf.keras.optimizers.Adam(lr=0.003, beta_1=.99)
    tf_optimizer_u = tf.keras.optimizers.Adam(lr=0.03, beta_1=.99)

    tf.print(f"weight_ub: {weight_ub}  weight_fu: {weight_fu}")
    print("starting Adam training")

    a = np.random.rand(1000)
    loss_history = list(a)
    MSE_b0 = list(a)
    MSE_f0 = list(a)

    MSE_b1 = []
    MSE_f1 = []

    weightu = []
    weightf = []
    # For mini-batch (if used)
    for epoch in range(tf_iter):
        for i in range(n_batches):
            xb_batch = initial[:, 1:2]
            yb_batch = initial[:, 2:3]
            tb_batch = initial[:, 0:1]
            ub_batch = initial[:, 3:4]
            vb_batch = initial[:, 4:5]
            pb_batch = initial[:, 5:6]

            x_f_batch = collo[:, 1:2][i * batch_sz:(i * batch_sz + batch_sz), ]
            y_f_batch = collo[:, 2:3][i * batch_sz:(i * batch_sz + batch_sz), ]
            t_f_batch = collo[:, 0:1][i * batch_sz:(i * batch_sz + batch_sz), ]

            loss_value, mse_b, mse_f, grads, grads_ub, grads_fu = grad(u_model, x_f_batch, y_f_batch, t_f_batch,
                                                                       xb_batch, yb_batch,
                                                                       tb_batch, ub_batch, vb_batch, weight_ub,
                                                                       weight_fu)

            tf_optimizer.apply_gradients(zip(grads, u_model.trainable_variables))
            MSE_b0.append(mse_b)
            MSE_f0.append(mse_f)

            loss_history.append(loss_value)
            
            if loss_history[-1] < loss_history[-2] and loss_history[-2] < loss_history[-3] and loss_history[-1] < loss_history[-10]:
                tf_optimizer_weights.apply_gradients(zip([-grads_fu], [weight_fu]))
                tf_optimizer_u.apply_gradients(zip([-grads_ub], [weight_ub]))

        if epoch % 10 == 0:
            elapsed = time.time() - start_time
            print('It: %d, Time: %.2f' % (epoch, elapsed))
            tf.print(f"mse_b  {mse_b}  mse_f: {mse_f}   total loss: {loss_value}")

            wu = weight_ub.numpy()
            wf = weight_fu.numpy()

            MSE_b1.append(mse_b)
            MSE_f1.append(mse_f)

            weightu.append(wu)
            weightf.append(wf)

            start_time = time.time()
    tf.print(f"weight_ub: {weight_ub}  weight_fu: {weight_fu}")
    u_pred, v_pred, p_pred = predict(X_star)
    error_u = np.linalg.norm(u_exact1 - u_pred, 2) / np.linalg.norm(u_exact1, 2)
    print('Error u: %e' % (error_u))
    error_v = np.linalg.norm(v_exact1 - v_pred, 2) / np.linalg.norm(v_exact1, 2)
    print('Error v: %e' % (error_v))
    print("Starting L-BFGS training")

    loss_and_flat_grad = get_loss_and_flat_grad(x_f_batch, y_f_batch, t_f_batch, xb_batch, yb_batch, tb_batch, ub_batch,  vb_batch, weight_ub, weight_fu)

    lbfgs(loss_and_flat_grad,  get_weights(u_model), Struct(), maxIter=newton_iter1, learningRate=0.8)

    u_pred, v_pred, p_pred = predict(X_star)
    error_u = np.linalg.norm(u_exact1 - u_pred, 2) / np.linalg.norm(u_exact1, 2)
    print('Error u: %e' % (error_u))
    error_v = np.linalg.norm(v_exact1 - v_pred, 2) / np.linalg.norm(v_exact1, 2)
    print('Error v: %e' % (error_v))

    error_p = np.linalg.norm(p_exact1 - p_pred, 2) / np.linalg.norm(p_exact1, 2)
    print('Error p: %e' % (error_p))


    # lbfgs(loss_and_flat_grad, get_weights(u_model), Struct(), maxIter=newton_iter2, learningRate=0.8)

    return MSE_b1, MSE_f1,  weightu, weightf



In [7]:
# L-BFGS implementation from https://github.com/pierremtb/PINNs-TF2.0
def get_loss_and_flat_grad(x_f_batch, y_f_batch, t_f_batch, xb_batch, yb_batch, tb_batch, ub_batch, vb_batch,weight_ub, weight_fu):
    def loss_and_flat_grad(w):
        with tf.GradientTape() as tape:
            set_weights(u_model, w, sizes_w, sizes_b)
            loss_value, _, _ = loss(x_f_batch, y_f_batch, t_f_batch, xb_batch, yb_batch, tb_batch, ub_batch, vb_batch, weight_ub, weight_fu)
        grad = tape.gradient(loss_value, u_model.trainable_variables)
        grad_flat = []
        for g in grad:
            grad_flat.append(tf.reshape(g, [-1]))
        grad_flat = tf.concat(grad_flat, 0)
        # print(loss_value, grad_flat)
        return loss_value, grad_flat

    return loss_and_flat_grad


def predict(X_star):
    X_star = tf.convert_to_tensor(X_star, dtype=tf.float32)
    u_star, v_star, p_star = u_x_model(X_star[:, 0:1], X_star[:, 1:2], X_star[:, 2:3])
    return u_star.numpy(), v_star.numpy(), p_star.numpy()


In [8]:
########################################################################################

def get_training_dataset(path , pINLET , pOUTLET , pWALL , pDomain , pInitial , dist):
    
    data = h5py.File(path , 'r')  # load dataset from matlab
    WALL = np.transpose(data['noslipwall_02_Z0slice_2D_wall'], axes=range(len(data['noslipwall_02_Z0slice_2D_wall'].shape) - 1,-1, -1)).astype(np.float32)
    
    WALL = np.delete(WALL , np.where(WALL[:,0] == WALL[:,0].min())[0] , 0)  
    # WALL = np.delete(WALL , np.where(WALL[:,1] <= 0.2)[0] , 0)  


    domain = np.transpose(data['noslipwall_02_Z0slice_2D_innerdomain'], axes=range(len(data['noslipwall_02_Z0slice_2D_innerdomain'].shape) - 1,-1, -1)).astype(np.float32)
    
    domain = np.delete(domain , np.where(domain[:,0] == domain[:,0].min())[0] , 0)  
    # domain = np.delete(domain , np.where(domain[:,1] <= 0.2)[0] , 0)  


    # INLET = np.transpose(data['noslipwall_02_Z0slice_2D_inlet'],  axes=range(len(data['noslipwall_02_Z0slice_2D_inlet'].shape) - 1,-1, -1)).astype(np.float32)
        
    INLET = domain[np.where(domain[:,1] == domain[:,1].min())[0],:] #np.delete(INLET , np.where(INLET[:,0] == INLET[:,0].min())[0] , 0)  

    OUTLET = np.transpose(data['noslipwall_02_Z0slice_2D_outlet'], axes=range(len(data['noslipwall_02_Z0slice_2D_outlet'].shape) - 1,-1, -1)).astype(np.float32)
    
    OUTLET = np.delete(OUTLET , np.where(OUTLET[:,0] == OUTLET[:,0].min())[0] , 0)  

    total = INLET.shape[0] + OUTLET.shape[0] + domain.shape[0] +  WALL.shape[0] 
    
    np.random.seed(1234)

    # initial domain ux is 0.2 and other values are zero. FOr wall, all values are zero
    
    INITIALd = domain[np.where(domain[:,0] == domain[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    INITIALw = WALL[np.where(WALL[:,0] == WALL[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    INITIALi = INLET[np.where(INLET[:,0] == INLET[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    INITIALo = OUTLET[np.where(OUTLET[:,0] == OUTLET[:,0].min())[0],:] # initial doamin corressponds to all values where t is zero

    #random selection of training data
    INITIAL = np.concatenate([INITIALd , INITIALo],0)
    
    # domain = np.concatenate([domain , WALL , INLET , OUTLET],0)

    #INLET = np.delete(INLET , idx_initi , 0)  
    #OUTLET = np.delete(OUTLET , idx_inito  , 0)  
    #domain = np.delete(domain , idx_initd , 0)  
    #WALL = np.delete(WALL , idx_initw , 0)  
    
    if dist == "Sobol":
        idxi = generate_sobol_sequence(0 , INLET.shape[0] ,  int(INLET.shape[0] * pINLET)) 
        INLET = INLET[idxi, :]
        idxi = generate_sobol_sequence(0 , OUTLET.shape[0] ,  int(OUTLET.shape[0] * pOUTLET)) 
        OUTLET = OUTLET[idxi, :]
        idxi = generate_sobol_sequence(0 , INITIAL.shape[0] ,  int(INITIAL.shape[0] * pInitial)) 
        INITIAL = INITIAL[idxi, :]
        idxi = generate_sobol_sequence(0 , WALL.shape[0] ,  int(WALL.shape[0] * pWALL)) 
        WALL = WALL[idxi, :]
        idxi = generate_sobol_sequence(0 , domain.shape[0] ,  int(domain.shape[0] * pDomain)) 
        domain = domain[idxi, :]
    else:
        idxi = np.random.choice(INLET.shape[0], int(INLET.shape[0] * pINLET), replace=False)
        INLET = INLET[idxi, :]
        idxi = np.random.choice(OUTLET.shape[0], int(OUTLET.shape[0] * pOUTLET), replace=False)
        OUTLET = OUTLET[idxi, :]
        idxi = np.random.choice(INITIAL.shape[0], int(INITIAL.shape[0] * pInitial), replace=False)
        INITIAL = INITIAL[idxi, :]
        idxi = np.random.choice(WALL.shape[0], int(WALL.shape[0] * pWALL), replace=False)
        WALL = WALL[idxi, :]
        idxi = np.random.choice(domain.shape[0], int(domain.shape[0] * pDomain), replace=False)
        domain = domain[idxi, :]

    #########################################
    return [domain , INLET , OUTLET, WALL, INITIAL , total]
################################################################

In [9]:
from generate_dataset import * 
# path = '/okyanus/users/afarea/dataDir/noslipwall_02_Z0slice_2D/noslipwall_02_Z0slice_2D.mat'
path = '/media/afrah2/MyWork/files2023/dataset/noslipwall_02_Z0slice_2D.mat'

modelPath = ''


pINLET = 1
pOUTLET= 0.3
pWALL =  0.006
pDomain = 0.0007
pInitial = 0.07

dist =  "Sobol"
activFun = 'hard_swish'

[collo , inlet , outlet, wall, initial , total] = get_training_dataset(path , pINLET , pOUTLET , pWALL , pDomain , pInitial , dist)

XY_c = np.concatenate([collo , wall, inlet , outlet], 0) 



# # Define model
mode = 'M1'

input_dimension = 3
output_dimension = 3
n_hidden_layers = 3
neurons = 50

starter_learning_rate = float(0.005)
epochs = 5
layers = [input_dimension] + n_hidden_layers*[neurons] + [output_dimension]

batch_size= 300

iterations = 40000

method  = "mini_batch"


###############################################################
# model = TwoD_BFS_Slip_PINN(XY_c , layers  , activFun , mode , starter_learning_rate , ExistModel = modelPath)


# model.print("Using mode: " , model.mode)
# model.print("neural network: " , model.layers )
# model.print("collocation Training data size : " ,collo.shape[0], " (" ,str(pDomain*100) , "%)")
# model.print("Wall Training data size: " ,wall.shape[0], " (" ,str(pWALL*100) , "%)")
# model.print("Initial Training data size: " ,initial.shape[0], " (" ,str(pInitial*100) , "%)")
# model.print("INLET Training data size: " ,inlet.shape[0], " (" ,str(pINLET*100) , "%)")
# model.print("OUTLET Training data size: " ,outlet.shape[0], " (" ,str(pOUTLET*100) , "%)")

# model.print("Total training datset size: " ,XY_c.shape[0], " (" ,str((XY_c.shape[0] / total)*100) , "%)")
# model.print("Activation function: " , activFun)
# model.print("number of iterations: " , iterations)

# model.print("Method desciption : learning rates: Exponential decay  with initial value: ", starter_learning_rate)


# model.print("File directory: " , model.dirname)

# #save_data_to_matlab(os.path.join(model.dirname,'slipwall_02_Z0slice_2D_training.mat') ,coll , INLET , OUTLET , WALL ,INITIAL )

# plot_dataset( model.dirname , wall , inlet , outlet , collo , initial , dist)




In [11]:

X_f = XY_c
X_u_train = wall



X_star1 = collo
x_star1 = np.array([X_star1[:, 0]])
y_star1 = np.array([X_star1[:, 1]])
t_star1 = np.array([X_star1[:, 2]])

u_exact1=np.array([X_star1[:, 3]])
v_exact1=np.array([X_star1[:, 4]])
p_exact1=np.array([X_star1[:, 5]])

xb = tf.cast(X_u_train[:, 0:1], dtype=tf.float32)
yb = tf.cast(X_u_train[:, 1:2], dtype=tf.float32)
tb = tf.cast(X_u_train[:, 2:3], dtype=tf.float32)
ub = tf.cast(X_u_train[:, 3:4], dtype=tf.float32)
vb = tf.cast(X_u_train[:, 4:5], dtype=tf.float32)


x_f = tf.convert_to_tensor(X_f[:, 0:1], dtype=tf.float32)
y_f = tf.convert_to_tensor(X_f[:, 1:2], dtype=tf.float32)
t_f = tf.convert_to_tensor(X_f[:, 2:3], dtype=tf.float32)

b = X_star1.min(0)
rb = X_star1.max(0)

start_time = time.time()

In [12]:
N_f = 10000
Nu1 = 200

weight_ub = tf.Variable([1.0], dtype=tf.float32)
weight_fu = tf.Variable([1.0], dtype=tf.float32)

MSE_b1, MSE_f1, weightu, weightf = fit(collo[::10,:], inlet, outlet, wall, initial,
                                       weight_ub, weight_fu,
                                         tf_iter=1000, 
                                         tf_iter2=100, 
                                         newton_iter1=500,
                                         newton_iter2=1500)




weight_ub: <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>  weight_fu: <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([1.], dtype=float32)>
starting Adam training


OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26060 thread 1 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26070 thread 3 bound to OS proc set 3
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26069 thread 2 bound to OS proc set 2
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26071 thread 4 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26073 thread 6 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26072 thread 5 bound to OS proc set 5
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26074 thread 7 bound to OS proc set 7
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26075 thread 8 bound to OS proc set 8
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26076 thread 9 bound to OS proc set 9
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26077 thread 10 bound to OS proc set 10
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26079 thread 12 bound to OS proc set 12
OMP: Info #254: KMP_AFFINITY: pid 26015 tid 26078 thread 11 bound to OS proc set 11
OMP: Info 

It: 0, Time: 30.96
mse_b  [16294.189]  mse_f: [82.16595]   total loss: [16376.355]
It: 10, Time: 1.71
mse_b  [5069.53]  mse_f: [6.1845226]   total loss: [5075.7144]
It: 20, Time: 1.36
mse_b  [2750.938]  mse_f: [22.093035]   total loss: [2773.031]
It: 30, Time: 1.79
mse_b  [85.14449]  mse_f: [11.838937]   total loss: [96.98343]
It: 40, Time: 1.65
mse_b  [1783.4163]  mse_f: [14.218702]   total loss: [1797.635]
It: 50, Time: 1.31
mse_b  [1726.6447]  mse_f: [6.1706295]   total loss: [1732.8153]
It: 60, Time: 1.49
mse_b  [98.84054]  mse_f: [12.554775]   total loss: [111.39531]
It: 70, Time: 1.52
mse_b  [1371.1953]  mse_f: [17.63553]   total loss: [1388.8308]
It: 80, Time: 1.31
mse_b  [354.00308]  mse_f: [15.53737]   total loss: [369.54047]
It: 90, Time: 1.48
mse_b  [834.614]  mse_f: [8.127516]   total loss: [842.7415]
It: 100, Time: 1.32
mse_b  [185.45494]  mse_f: [15.68756]   total loss: [201.1425]
It: 110, Time: 1.67
mse_b  [775.36694]  mse_f: [17.570923]   total loss: [792.93787]
It: 120

In [16]:

elapsed = time.time() - start_time
print('Training time: %.4f' % (elapsed))

u_pred, v_pred, p_pred = predict(X_star1)

# U_pred = u_pred.reshape((x1.shape[0], y1.shape[0], t1.shape[0]))
# V_pred = v_pred.reshape((x1.shape[0], y1.shape[0], t1.shape[0]))
# P_pred = p_pred.reshape((x1.shape[0], y1.shape[0], t1.shape[0]))
u_exact1 = np.array([X_star1[:, 3]])
v_exact1 = np.array([X_star1[:, 4]])
p_exact1 = np.array([X_star1[:, 5]])


error_uu = np.abs( u_exact1 - u_pred)
error_vv = np.abs(v_exact1 - v_pred)
error_pp = np.abs( p_exact1 - p_pred)

error_u = np.linalg.norm(u_exact1 - u_pred, 2) / np.linalg.norm(u_exact1, 2)
print('Error u: %e' % (error_u))

error_v = np.linalg.norm(v_exact1 - v_pred, 2) / np.linalg.norm(v_exact1, 2)
print('Error v: %e' % (error_v))

error_p = np.linalg.norm(p_exact1 - p_pred, 2) / np.linalg.norm(p_exact1, 2)
print('Error p: %e' % (error_p))

# dataNewNS = 'NS_hisyory.mat'
# scipy.io.savemat(dataNewNS, {'w_MSE_b': MSE_b1, 'w_MSE_f': MSE_f1, 'weight_u': weightu,
#                   'weight_f': weightf, 'U_pred': u_pred, 'V_pred': V_pred, 'P_pred': P_pred})

Training time: 1488.1751
Error u: 4.266133e+01
Error v: 4.266161e+01
Error p: 8.245512e+01
